# Case 5 -- Steady Constant Sea Level (from Joe)

Comparison with MODFLOW-USG results
Does NOT use Newton formulation

In [ ]:
import pathlib as pl
import numpy as np
import flopy

# path to mf6 shared library
# executables based on mf6 feat-swi branch at: 
#   https://github.com/langevin-usgs/modflow6/tree/feat-swi-correction

# Put the name of the mf6 executable into mf6exe.txt,
# which is not under version control.
with open(pl.Path("./mf6exe.txt"), "r") as f:
    mf6exe = f.readline().strip()
print(f"using executable: {mf6exe}")

sim_ws = pl.Path("./temp/case5")

## MODFLOW-USG results for problem

In [ ]:
hds_usg = np.array(
      [0.2       , 0.59686235, 0.76508142, 0.89446596, 1.00096258,
       1.0914691 , 1.16976336, 1.23818961, 1.29832132, 1.35127281,
       1.397864  , 1.43871521, 1.47430508, 1.50500784, 1.53111814,
       1.55286808, 1.57043912, 1.58397057, 1.59356545, 1.59929471,
       1.60119991, 1.59929471, 1.59356545, 1.58397057, 1.57043912,
       1.55286808, 1.53111814, 1.50500784, 1.47430508, 1.43871521,
       1.397864  , 1.35127281, 1.29832132, 1.23818961, 1.16976336,
       1.0914691 , 1.00096258, 0.89446596, 0.76508142, 0.59686235,
       0.2       ]
)

In [ ]:
zeta_usg = np.array(
    [ -8.        , -23.87449386, -30.60325662, -35.77863827,
       -40.03850338, -43.65876407, -46.79053437, -49.52758447,
       -51.93285291, -54.05091221, -55.91456001, -57.5486084 ,
       -58.97220318, -60.20031357, -61.24472552, -62.11472301,
       -62.81756499, -63.35882268, -63.74261796, -63.97178855,
       -64.04799631, -63.97178855, -63.74261796, -63.35882268,
       -62.81756499, -62.11472301, -61.24472552, -60.20031357,
       -58.97220318, -57.5486084 , -55.91456001, -54.05091221,
       -51.93285291, -49.52758447, -46.79053437, -43.65876407,
       -40.03850338, -35.77863827, -30.60325662, -23.87449386,
        -8.        ]
)

In [ ]:
hds_usg.shape

## Create and Run Simple Test Model

In [ ]:
#create simple test model
Lx = 2050.0 # meters
ncol = 41
nlay = 1
nrow = 1
delr, delc = Lx / ncol, 1.0
top = 5.
botm = -80.
recharge = 1e-3
k = 10.
h0 = 0.2  
h1 = h0
icelltype = 1
newton_option = None
max_outer = 50
if newton_option is None:
    linear_accel = "cg"
else:
    linear_accel = "bicgstab"

name = "usg"
sim = flopy.mf6.MFSimulation(
    sim_name=name, 
    sim_ws=sim_ws, 
    exe_name=mf6exe,
    memory_print_option="all"
)
tdis = flopy.mf6.ModflowTdis(sim)
ims = flopy.mf6.ModflowIms(
    sim, 
    print_option="all",
    outer_dvclose=1e-8, 
    inner_dvclose=1e-9,
    linear_acceleration=linear_accel, 
    outer_maximum=max_outer,
)
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True, newtonoptions=newton_option)
dis = flopy.mf6.ModflowGwfdis(
    gwf, 
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)
ic = flopy.mf6.ModflowGwfic(gwf)
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    save_specific_discharge=True,
    save_saturation=True,
    # alternative_cell_averaging="AMT-HMK",
    icelltype=icelltype,
    k=k,
)
zeta_file = name + '.zta'
# swi = flopy.mf6.ModflowGwfswi(gwf, zeta_filerecord=zeta_file, zetastrt=botm)
swi = flopy.mf6.ModflowGwfswi(gwf, zeta_filerecord=zeta_file, zetastrt=-1)
chd_spd = [
    ((0, 0, 0), h0),
    ((0, 0, ncol - 1), h1),
]
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_spd)
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=recharge)
budget_file = name + '.bud'
head_file = name + '.hds'
oc = flopy.mf6.ModflowGwfoc(
        gwf,
        budget_filerecord=budget_file,
        head_filerecord=head_file,
        saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
        printrecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
)
sim.write_simulation()
sim.run_simulation()

In [ ]:
fpth = sim_ws / f"{gwf.name}.zta"
zeta_all = flopy.utils.HeadFile(fpth, text="zeta").get_alldata()
zeta = zeta_all[-1].flatten()

x = gwf.modelgrid.xcellcenters.flatten()
head = gwf.output.head().get_data().flatten()
bud = gwf.output.budget()
#print(head.flatten())
#print(zeta.flatten())

spdis = bud.get_data(text='DATA-SPDIS')[0]
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)
pxs = flopy.plot.PlotCrossSection(gwf, line={"row":0})
ax = pxs.ax
if ncol < 50:
    pxs.plot_grid()
#pxs.plot_bc("ghb")
ax.plot(x, head, "k-")
ax.plot(x, zeta, "k--")
# freshwater
ax.fill_between(x, head, zeta, color="cyan")
# saltwater
ax.fill_between(x, zeta, botm, color="red")
# pxs.plot_vector(qx, qy, qz, normalize=True, color="black")

ax.plot(x, hds_usg, lw=0, mec="black", mfc="none", marker="o")
ax.plot(x, zeta_usg, lw=0, mec="black", mfc="none", marker="x");


## Plot Results

In [ ]:
fpth = sim_ws / f"{name}.dis.grb"
grb = flopy.mf6.utils.MfGrdFile(fpth)
ia = grb.ia
ja = grb.ja
nodes = grb.nodes
flowja = bud.get_data(text="FLOW-JA-FACE")[0].flatten()

# print flows bewteen cells
# for n in range(nodes):
#     for ipos in range(ia[n] + 1, ia[n + 1]):
#         m = ja[ipos]
#         q = flowja[ipos]
#         if m > n:
#             print(f"{n=} {m=} {q=}")

In [ ]:
bobj = gwf.output.budget()

In [ ]:
bobj.get_unique_record_names()

In [ ]:
bobj.get_data(text="DATA-SAT")

In [ ]:
-head * 40.

In [ ]:
zeta

In [ ]:
(-head * 40.) - zeta

In [ ]:
head.max()